In [1]:
from gensim.models import KeyedVectors, Word2Vec
import numpy as np

In [2]:
import pandas as pd
csv_file = "/Users/i337036/Downloads/train.csv"
df = pd.read_csv(csv_file, encoding='utf-8')
len(df)

404290

In [3]:
df.columns.values

array(['id', 'qid1', 'qid2', 'question1', 'question2', 'is_duplicate'],
      dtype=object)

In [4]:
df = df[df['is_duplicate']==1]

In [5]:
df = df.drop_duplicates(subset=['question1'], keep='first')
df = df.drop_duplicates(subset=['question2'], keep='first')
len(df)

73776

In [6]:
len((df["question1"] + df["question2"]).unique())

73776

In [7]:
q1_list = df.question1.tolist()
q2_list = df.question2.tolist()
print(len(q1_list), len(q2_list))

73776 73776


In [8]:
def clean_sentence(sentence):
    sentence = [word for word in sentence.strip().split() if word in word_vecs.wv.vocab]
    
    return sentence

In [9]:
def get_sentence_matrix(sentences):
    sentence_mat = np.zeros((len(sentences), 300), dtype=float)
    for idx in tqdm(range(len(sentences)), desc="Building vectors for %d sentences" % len(sentences)):
        sentence = sentences[idx]
        for word in sentence:
            sentence_mat[idx, :] += word_vecs[word]
            
    return sentence_mat

In [10]:
from tqdm import tqdm

In [11]:
train_sentences = q1_list + q2_list
print(len(train_sentences))

147552


In [12]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [13]:
word_vecs = Word2Vec(min_count=1, size=300, workers=8, seed=42, iter=50)
word_vecs.build_vocab(train_sentences)
word_vecs.train(sentences=train_sentences, total_examples=word_vecs.corpus_count, epochs=word_vecs.epochs)

2018-04-24 15:48:58,146 : INFO : collecting all words and their counts
2018-04-24 15:48:58,147 : WARNING : Each 'sentences' item should be a list of words (usually unicode strings). First item here is instead plain <class 'str'>.
2018-04-24 15:48:58,148 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-04-24 15:48:58,205 : INFO : PROGRESS: at sentence #10000, processed 518975 words, keeping 127 word types
2018-04-24 15:48:58,255 : INFO : PROGRESS: at sentence #20000, processed 1037154 words, keeping 153 word types
2018-04-24 15:48:58,308 : INFO : PROGRESS: at sentence #30000, processed 1556651 words, keeping 175 word types
2018-04-24 15:48:58,362 : INFO : PROGRESS: at sentence #40000, processed 2079699 words, keeping 184 word types
2018-04-24 15:48:58,416 : INFO : PROGRESS: at sentence #50000, processed 2595725 words, keeping 199 word types
2018-04-24 15:48:58,470 : INFO : PROGRESS: at sentence #60000, processed 3112930 words, keeping 217 word types
2018-0

(80110156, 382727600)

In [14]:
q1_word_list = [clean_sentence(txt1) for txt1 in q1_list]
q2_word_list = [clean_sentence(txt2) for txt2 in q2_list]

In [15]:
q2_mat = get_sentence_matrix(q1_word_list)
q1_mat = get_sentence_matrix(q2_word_list)

Building vectors for 73776 sentences: 100%|██████████| 73776/73776 [00:00<00:00, 132332.47it/s]


In [16]:
from sklearn.metrics.pairwise import cosine_similarity

def find_similar(q1_mat, q2_mat, index, top_n=5):
    qd_sims = cosine_similarity(q1_mat[index: index+1], q2_mat).flatten()
    qd_indices = qd_sims.argsort()[:-top_n:-1]
    return qd_indices

In [17]:
from random import randint

In [18]:
search_idx = randint(0, len(q1_list))
print("Searched for: %s at [%d]" % (q1_list[search_idx], search_idx))
for index in find_similar(q1_mat, q2_mat, search_idx):
    print ("%s %d" % (q2_list[index], index))

Searched for: What is the best place for a visit in December in India? at [61701]
What does Jainism say about Gays and Homosexuality? 73775
If I have dual citizenship, when immigration scans one passport, can they see that I have another passport and any other information? 24207
What's the meaning of the name in other countries? 24587
What is the use of tanpura in music? 24588


In [19]:
q1_list[search_idx], q2_list[search_idx]

('What is the best place for a visit in December in India?',
 'What are the best places to visit in India in December?')

In [20]:
from tqdm import tqdm

In [21]:
correct_index = 0
for search_idx in tqdm(range(500)):
    for index in find_similar(q1_mat, q2_mat, search_idx):
        if search_idx==index or q1_list[search_idx]==q2_list[index]:
            correct_index += 1
            break

100%|██████████| 500/500 [01:41<00:00,  4.92it/s]


In [22]:
print(correct_index)

3
